## Analyzing dome monitor with a labJack T7
Uses 2 analog input (AINs) to read the data.

Craig Lage - Jul 5, 2023

In [ ]:
import sys, os, re
import time, datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
dir = "/scratch/labJackData/"

files = os.listdir(dir)
counter = 0
for file in files:
    split_file = re.split('_|T|\Z.', file)
    if split_file[0] == "Dome":
        if (split_file[1] == '20230818') or \
            (split_file[1] == '20230817' and int(list(split_file[2])[0]) > 0):
            filename = dir+file

            pkl_file = open(filename, 'rb')
            df = pkl.load(pkl_file)
            pkl_file.close()
            print(filename, len(df), df.columns)
            if counter == 0:
                merged_df = df
            else:
                merged_df = pd.concat([merged_df, df], axis=1)
            counter += 1


In [ ]:
client = EfdClient('summit_efd')
start = Time("2023-08-17 21:30:00Z", scale='utc')
end = Time("2023-08-18 06:00:00Z", scale='utc')
state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)


In [ ]:
len(merged_df)

In [ ]:
fig, ax = plt.subplots(1,1)
ax2 = ax.twinx()
state['state'].plot(ax=ax2, color='blue', label='Actual State')
ax2.set_ylim(0,6)
ax2.set_yticks([1,2,3,4,5])
ax2.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing'])


merged_df["AIN0"].plot(ax=ax, label="Monitor_0", color='red')
merged_df["AIN1"].plot(ax=ax, label="Monitor_1", color='green')
ax.set_ylim(-0.5,10.0)
ax.set_title("AuxTel Dome Closing Monitor 20230817", fontsize=16)

ax.get_legend().remove()
plt.savefig(f"/home/craiglagegit/DATA/Dome_Monitor_17Aug23.png")

In [ ]:
start_plot = Time("2023-08-18 01:30:00Z", scale='utc')
end_plot = Time("2023-08-18 01:32:00Z", scale='utc')
sub_df = merged_df.loc[(merged_df.index >= start_plot.isot) & (merged_df.index <= end_plot.isot)]

In [ ]:
len(sub_df)

In [ ]:
fig, ax = plt.subplots(1,1)
ax2 = ax.twinx()
state['state'].plot(ax=ax2, color='blue', label='Actual State')
ax2.set_ylim(0,6)
ax2.set_yticks([1,2,3,4,5])
ax2.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing'])


sub_df["AIN0"].plot(ax=ax, label="Monitor_0", color='red')
sub_df["AIN1"].plot(ax=ax, label="Monitor_1", color='green')
ax.set_title("AuxTel Dome Closing Monitor 20230817", fontsize=16)
ax.set_ylim(-0.5,10.0)
ax.text("2023-08-18 01:30:30Z", 1.0, "AIN0", color='red')
ax.text("2023-08-18 01:30:45Z", 8.0, "AIN1", color='green')
ax.set_xlim(start_plot.isot, end_plot.isot)
ax.get_legend().remove()
plt.savefig(f"/home/craiglagegit/DATA/Dome_Monitor_BlowUp_17Aug23.png")